## Search Engine Development - TF * IDF

### Importing Modules

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# reading the json data into a dataframe

items = pd.read_json("./Datasets/Processed/itemDetails_RatingCount_gt15.json")

In [3]:
items.shape

(1302659, 18)

In [4]:
items.head()

,isbn,average_rating,similar_books,description,link,authors,publisher,num_pages,publication_day,publication_month,publication_year,isbn13,url,image_url,book_id,ratings_count,title,mod_title
0,,4.03,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600,,,1987,,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",the unschooled wizard sun wolf and starhawk 12
1,0743294297,3.49,"[6604176, 6054190, 2285777, 82641, 7569453, 70...",Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",Atria Books,368,14,7,2009,9780743294294,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,Best Friends Forever,best friends forever
2,1599150603,4.13,[],"Relates in vigorous prose the tale of Aeneas, ...",https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,162,13,9,2006,9781599150604,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,The Aeneid for Boys and Girls,the aeneid for boys and girls
3,1934876569,4.22,"[948696, 439885, 274955, 12978730, 372986, 216...","To Kara's astonishment, she discovers that a p...",https://www.goodreads.com/book/show/6066812-al...,"[{'author_id': '19158', 'role': ''}]",Seven Seas,216,3,3,2009,9781934876565,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,98,All's Fairy in Love and War (Avalon: Web of Ma...,alls fairy in love and war avalon web of magic 8
4,0922915113,3.81,"[287151, 1104760, 1172822, 440292, 287082, 630...","Wisdom, humor, and dark observations by the fo...",https://www.goodreads.com/book/show/287149.The...,"[{'author_id': '2983296', 'role': ''}, {'autho...",Feral House,147,1,4,2000,9780922915118,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,287149,986,The Devil's Notebook,the devils notebook


### TF X IDF Calculation - mod_title Column - Unigrams

In [45]:
# initializing the object
vectorizer_uni = TfidfVectorizer()

# generating the terms and calculating the tf*idf score
tfidf_uni = vectorizer_uni.fit_transform(items["mod_title"])

In [46]:
# shape of the total tfidf matrix
# row, columns = movies, terms

tfidf_uni.shape

(1302659, 233827)

### Calculating Similarity and Preparing the Matrix - Cosine Similarity

In [47]:
# in pandas we can use style to display things using html
# making the title clickable by attaching a link 
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val)

# showing the image by attaching a link
def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

In [56]:
# creating the search function
# search function will take => user query and the vectorizer as input
 
def search_uni(query,vectorizer=vectorizer_uni):
    # normalizing the query string => removing unnecessary characters and making lower case
    processed = re.sub("\s+", " ", re.sub("[^a-zA-Z0-9 ]", "", query.lower()))

    # converting query string into vector using the vectorizer
    query_vec = vectorizer.transform([processed])
    # one row and 233827 columns => (number of columns = number of terms)
    print(query_vec.shape)
    
    # checking the similarity between query vector and each vector present in tfidf matrix
    # (1302659, 233827) , (1, 233827)
    similarity = cosine_similarity(query_vec, tfidf_uni)
    # we will get similarity value between query string vector and each of the tfidf matrix vectors
    # So, in total we will get 1302659 similarity values as (1, 1302659) vector
    print(similarity.shape)
    # We can then flatten it to 1D
    similarity = similarity.flatten()
    print(similarity.shape)

    # we are getting 10 largest similarities
    # Actually, we are getting their index using argpartition
    indices = np.argpartition(similarity, -50)[-50:]

    # we can use the indices to get the corresponding item/book details
    # So, we will have the item/book records corresponding to selected indices   
    results = items.iloc[indices]

    # we will sort the books based on highest ratings_count
    results = results.sort_values("ratings_count", ascending=False)
    
    # returning the top results with specified style
    # Note: we are using only some columns for demonstration
    columns = ["book_id", "title", "ratings_count", "url", "image_url", "mod_title"]

    # removing duplicate results based on mod_title column
    unique_results = results.drop_duplicates(subset=['mod_title'])

    return unique_results[columns].head().style.format({'url': make_clickable, 'image_url': show_image}) 

### Test the Search Engine

In [62]:
search_uni("Devil's Advocate")

(1, 233827)
(1, 1302659)
(1302659,)


,book_id,title,ratings_count,url,image_url,mod_title
208004,248760,The Devil's Advocate,5777,Goodreads,,the devils advocate
1038552,15780350,The Advocate,3765,Goodreads,,the advocate
955592,15784613,"The Devil's Metal (Devils, #1)",2217,Goodreads,,the devils metal devils 1
746979,18684303,The Advocate's Betrayal (The Advocate #2),2063,Goodreads,,the advocates betrayal the advocate 2
124623,538064,The Devil's Love,1997,Goodreads,,the devils love


### Export the Vectorizer and TfIDF  - Unigram

In [42]:
import joblib

In [51]:
joblib.dump(vectorizer_uni, 'vectorizer_uni.pkl', compress=True)
joblib.dump(tfidf_uni, 'tfidf_matrix_uni.pkl', compress=True)

['tfidf_matrix_uni.pkl']

### TF X IDF Calculation - mod_title Column - Unigrams and Bigrams

In [52]:
# initializing the object
# ngram_range=(1, 2) => Unigram and Bigram

vectorizer_uni_bi = TfidfVectorizer(analyzer='word', ngram_range=(1, 2),
                     min_df=0, stop_words='english')

# generating the terms and calculating the tf*idf score
tfidf_uni_bi = vectorizer_uni_bi.fit_transform(items["mod_title"])

In [53]:
# shape of the total tfidf matrix
# row, columns = movies, terms

tfidf_uni_bi.shape

(1302659, 1828619)

### Calculating Similarity and Preparing the Matrix - Cosine Similarity

In [54]:
# in pandas we can use style to display things using html
# making the title clickable by attaching a link 
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val)

# showing the image by attaching a link
def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

In [58]:
# creating the search function
# search function will take => user query and the vectorizer as input
 
def search_bi(query,vectorizer=vectorizer_uni_bi):
    # normalizing the query string => removing unnecessary characters and making lower case
    processed = re.sub("\s+", " ", re.sub("[^a-zA-Z0-9 ]", "", query.lower()))

    # converting query string into vector using the vectorizer
    query_vec = vectorizer.transform([processed])
    # one row and 233827 columns => (number of columns = number of terms)
    print(query_vec.shape)
    
    # checking the similarity between query vector and each vector present in tfidf matrix
    # (1302659, 233827) , (1, 233827)
    similarity = cosine_similarity(query_vec, tfidf_uni_bi)
    # we will get similarity value between query string vector and each of the tfidf matrix vectors
    # So, in total we will get 1302659 similarity values as (1, 1302659) vector
    print(similarity.shape)
    # We can then flatten it to 1D
    similarity = similarity.flatten()
    print(similarity.shape)

    # we are getting 10 largest similarities
    # Actually, we are getting their index using argpartition
    indices = np.argpartition(similarity, -50)[-50:]

    # we can use the indices to get the corresponding item/book details
    # So, we will have the item/book records corresponding to selected indices   
    results = items.iloc[indices]

    # we will sort the books based on highest ratings_count
    results = results.sort_values("ratings_count", ascending=False)
    
    # returning the top results with specified style
    # Note: we are using only some columns for demonstration
    columns = ["book_id", "title", "ratings_count", "url", "image_url", "mod_title"]

    # removing duplicate results based on mod_title column
    unique_results = results.drop_duplicates(subset=['mod_title'])

    return unique_results[columns].head().style.format({'url': make_clickable, 'image_url': show_image}) 

### Test the Search Engine

In [63]:
search_bi("Devil's Advocate")

(1, 1828619)
(1, 1302659)
(1302659,)


,book_id,title,ratings_count,url,image_url,mod_title
208004,248760,The Devil's Advocate,5777,Goodreads,,the devils advocate
1038552,15780350,The Advocate,3765,Goodreads,,the advocate
710604,95656,The Ten Faces of Innovation: IDEO's Strategies for Defeating the Devil's Advocate and Driving Creativity Throughout Your Organization,2544,Goodreads,,the ten faces of innovation ideos strategies for defeating the devils advocate and driving creativity throughout your organization
746979,18684303,The Advocate's Betrayal (The Advocate #2),2063,Goodreads,,the advocates betrayal the advocate 2
38344,22006020,"The Devil's Fire (Devil's Fire, #1)",488,Goodreads,,the devils fire devils fire 1


### Export the Vectorizer and TfIDF Matrix - Unigrams and Bigrams

In [ ]:
import joblib

In [64]:
joblib.dump(vectorizer_uni, 'vectorizer_uni_bi.pkl', compress=True)
joblib.dump(tfidf_uni, 'tfidf_matrix_uni_bi.pkl', compress=True)

['tfidf_matrix_uni_bi.pkl']